In [1]:
import numpy as np
import os

In [2]:
os.add_dll_directory(r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8\bin")
import ibatensor

In [5]:
def print_tensor_info(label, tensor):
    print(f"{label} shape:", tensor.shape)
    s = tensor.to_string()
    print(s)

In [6]:
data = np.arange(25, dtype=np.float32).reshape(1, 1, 5, 5)
input_tensor = ibatensor.Tensor(data, 1)
print_tensor_info("Input Tensor", input_tensor)

# Create a 1x1x3x3 kernel (out_channels=1, in_channels=1, 3x3)
kernel_data = np.ones((1, 1, 3, 3), dtype=np.float32)
kernel_tensor = ibatensor.Tensor(kernel_data, 1)
print_tensor_info("Kernel", kernel_tensor)

print("\n🧪 Testing conv2d")
conv_output = input_tensor.conv2d(kernel_tensor, padding=1, stride=1)
print_tensor_info("Conv2D Output", conv_output)

print("\n🧪 Testing ReLU")
relu_out = conv_output.relu()
print_tensor_info("ReLU Output", relu_out)

print("\n🧪 Testing avg_pool")
avg_pooled = conv_output.avg_pool(K=2, padding=0, stride=2)
print_tensor_info("Avg Pool Output", avg_pooled)

print("\n🧪 Testing max_pool")
max_pool_result = conv_output.max_pool(K=2, padding=0, stride=2)
print_tensor_info("max_pool", max_pool_result.output)

print("\n🔁 Testing backprop functions")

# Use conv_output as sigma
sigma = conv_output

print("🔧 conv2d_backward_wr_kernel")
grad_kernel = ibatensor.conv2d_backward_wr_kernel(input_tensor, sigma, kernel_tensor, padding=1, stride=1)
print_tensor_info("Grad Kernel", grad_kernel)

print("🔧 conv2d_backward_wr_input")
grad_input = ibatensor.conv2d_backward_wr_input(input_tensor, sigma, kernel_tensor, padding=1, stride=1)
print_tensor_info("Grad Input", grad_input)

print("🔧 max_pool_backward_wr_input")
grad_mp = ibatensor.max_pool_backward_wr_input(conv_output, sigma, max_pool_result.max_inds_ptr, K=2, padding=0, stride=2)
print_tensor_info("Grad Max Pool", grad_mp)

print("🔧 avg_pool_backward_wr_input")
grad_avg = ibatensor.avg_pool_backward_wr_input(conv_output, sigma, K=2, padding=0, stride=2)
print_tensor_info("Grad Avg Pool", grad_avg)

print("🔧 conv2d_backwards_bias_wr_sigma")
grad_bias = ibatensor.conv2d_backwards_bias_wr_sigma(sigma)
print_tensor_info("Grad Bias", grad_bias)

print("🔧 relu_backwards")
grad_relu = ibatensor.relu_backwards(sigma, conv_output)
print_tensor_info("Grad ReLU", grad_relu)

print("🔧 bias_backwards")
grad_b = ibatensor.bias_backwards(sigma)
print_tensor_info("Bias Backward", grad_b)

print("\n✅ All tests complete!")

Input Tensor shape: [1, 1, 5, 5]

=== N=0, C=0 ===
0	1	2	3	4	
5	6	7	8	9	
10	11	12	13	14	
15	16	17	18	19	
20	21	22	23	24	


Kernel shape: [1, 1, 3, 3]

=== N=0, C=0 ===
1	1	1	
1	1	1	
1	1	1	



🧪 Testing conv2d
Conv2D Output shape: [1, 1, 5, 5]

=== N=0, C=0 ===
12	21	27	33	24	
33	54	63	72	51	
63	99	108	117	81	
93	144	153	162	111	
72	111	117	123	84	



🧪 Testing ReLU
ReLU Output shape: [1, 1, 5, 5]

=== N=0, C=0 ===
12	21	27	33	24	
33	54	63	72	51	
63	99	108	117	81	
93	144	153	162	111	
72	111	117	123	84	



🧪 Testing avg_pool
Avg Pool Output shape: [1, 1, 2, 2]

=== N=0, C=0 ===
30	48.75	
99.75	135	



🧪 Testing max_pool
max_pool shape: [1, 1, 2, 2]

=== N=0, C=0 ===
54	72	
144	162	



🔁 Testing backprop functions
🔧 conv2d_backward_wr_kernel
Grad Kernel shape: [1, 1, 3, 3]

=== N=0, C=0 ===
16632	20616	18123	
25068	30444	26316	
22374	26898	23028	


🔧 conv2d_backward_wr_input
Grad Input shape: [1, 1, 5, 5]

=== N=0, C=0 ===
120	210	270	270	276	
282	480	594	576	567	
486	810	972	918	822	
582